# 任务1 中文情感分类
使用数据集：ChnSentiCorp  
数据集介绍：情感分类数据集，每条数据包括一个购物评价，和一个标识，评价的商品包括书籍、酒店、计算机等。


In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
dataset = load_dataset("lansinuote/ChnSentiCorp")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})

## 模型架构
自然语言处理网络结构 RNN，RNN功能主要是把自然语言的句子抽取成特征向量，有了特征向量后再接入神经网络做分类或回归任务。RNN 把自然语言处理任务转换成全连接神经网络任务。对于类似RNN 这样能够把抽象数据类型转换成具体的特征向量的网络层，被称为 backbone，中文特征抽取层。

## Bert 模型训练下游任务
- 方式1 直接接入下游任务层

- 方式2 利用少量标注数据进行fine-tune


---
[参考网址](https://ifwind.github.io/2021/08/24/BERT%E7%9B%B8%E5%85%B3%E2%80%94%E2%80%94%EF%BC%887%EF%BC%89%E6%8A%8ABERT%E5%BA%94%E7%94%A8%E5%88%B0%E4%B8%8B%E6%B8%B8%E4%BB%BB%E5%8A%A1/)  
[bert 解决下游任务](https://zhuanlan.zhihu.com/p/365357615)

### 准备数据集
#### 1. 使用编码工具

In [2]:
# !pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.0 MB/s eta 0:00:00


In [6]:
!nvidia-smi
!nvcc -V

Fri Aug  4 05:50:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [8]:
from transformers import BertTokenizer
token = BertTokenizer.from_pretrained("bert-base-chinese")
token

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

编码器信息：
- vocab_size 词典大小 21128
- model_max_length 模型接收 token 最大长度512
- is_fast 速度慢
- padding_side 有填充
- truncation_side 有截断
- special_tokens 特殊字符 {'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}

In [9]:
out = token.batch_encode_plus(
    batch_text_or_text_pairs = ['海贼王','路飞','为了正义'],
    retrun_tensors = 'pt',
    return_length = True
)

for k,v in out.items():
  print(k,":",v)

print(token.decode(out['input_ids'][0]))

Keyword arguments {'retrun_tensors': 'pt'} not recognized.
Keyword arguments {'retrun_tensors': 'pt'} not recognized.
Keyword arguments {'retrun_tensors': 'pt'} not recognized.


input_ids : [[101, 3862, 6592, 4374, 102], [101, 6662, 7607, 102], [101, 711, 749, 3633, 721, 102]]
token_type_ids : [[0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
length : [5, 4, 6]
attention_mask : [[1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]
[CLS] 海 贼 王 [SEP]


#### 2.定义数据集

In [8]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [25]:
import torch
from datasets import load_from_disk
class Dataset(torch.utils.data.Dataset):
  def __init__(self, split):
    # already putting the dataset into google cloud drive, use google cloud drive
    self.dataset = load_from_disk('/content/drive/MyDrive/ChnSentiCorp')[split]
  def __len__(self):
    return len(self.dataset)
  def __getitem__(self, i):
    text = self.dataset[i]['text']
    label = self.dataset[i]['label']
    return text, label

testDataset = Dataset('test')
validationDataset = Dataset('validation')
trainDataset = Dataset('train')
len(trainDataset), trainDataset[20], trainDataset[3]

(9600,
 ('非常不错，服务很好，位于市中心区，交通方便，不过价格也高！', 1),
 ('1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.', 0))

#### 3. 定义计算设备

In [11]:
device = 'cpu'
if torch.cuda.is_available():
  device = 'cuda'
device

'cuda'

#### 4. 定义数据整理函数

In [12]:
def collate_fn(data):
  sents = [i[0] for i in data]
  labels = [i[1] for i in data]
  # encode
  data = token.batch_encode_plus(
      batch_text_or_text_pairs = sents,
      truncation = True,
      padding = "max_length",
      max_length = 512,
      return_tensors = 'pt',
      return_length = True
  )
  # move data into device
  input_ids = data['input_ids']
  token_type_ids = data['token_type_ids']
  attention_mask = data['attention_mask']
  labels = torch.LongTensor(labels)

  input_ids = input_ids.to(device)
  token_type_ids = token_type_ids.to(device)
  attention_mask = attention_mask.to(device)
  labels = labels.to(device)
  return input_ids, attention_mask, token_type_ids, labels

模拟数据，测试函数 collate_fn

In [13]:
fake_data = [
    ('欸有不错呦',1),
    ('从未见过有如此厚颜无耻之人',0),
    ('系好安全带准备漂移',1),
]
var1, var2, var3, var4 = collate_fn(fake_data)
var1.shape, var2.shape, var3.shape, var4

(torch.Size([3, 512]),
 torch.Size([3, 512]),
 torch.Size([3, 512]),
 tensor([1, 0, 1], device='cuda:0'))

#### 5. 定义数据集加载器
批量处理数据集中的数据

In [14]:
loader = torch.utils.data.DataLoader(
    dataset = trainDataset,
    batch_size = 20,
    collate_fn = collate_fn,
    shuffle = True,
    drop_last = True,
)
len(loader)
# show one batch data
for i,(var1, var2, var3, var4) in enumerate(loader):
  break
var1.shape, var2.shape, var3.shape, var4

(torch.Size([20, 512]),
 torch.Size([20, 512]),
 torch.Size([20, 512]),
 tensor([1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1],
        device='cuda:0'))

### 定义模型
#### 1. 加载预训练模型

In [15]:
from transformers import BertModel
pretrained = BertModel.from_pretrained("bert-base-chinese")
sum(i.nelement() for i in pretrained.parameters()) / 10000

10226.7648

模型参数超过 1个亿，本次只是将模型作为特征提取器，不对模型进行微调，需要冻结模型

In [16]:
for param in pretrained.parameters():
  param.requires_grad_(False)

对冻结模型进行试算，观察输入输出

In [17]:
pretrained.to(device)
out = pretrained(input_ids = var1,
        attention_mask = var2,
        token_type_ids = var3
)
print(out)
out.last_hidden_state.shape

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.2358, -0.2300, -0.7282,  ..., -0.6468, -0.0343, -0.0248],
         [ 1.3533, -0.1660,  0.6419,  ..., -1.0625, -0.6525,  0.3300],
         [ 1.1841, -0.9598,  0.0753,  ...,  0.4893,  0.5953, -0.3058],
         ...,
         [ 0.0903,  0.2679, -0.4704,  ...,  0.0959,  0.2613, -0.5820],
         [ 0.1733, -0.0822, -0.2382,  ..., -0.2566, -0.2432, -0.7103],
         [ 0.0993,  0.1186, -0.0062,  ..., -0.0862, -0.0162, -0.2260]],

        [[ 0.7697,  0.7899, -0.1347,  ..., -0.9141, -0.1910, -0.3087],
         [ 0.2831,  0.2378, -0.7799,  ..., -0.3764,  0.4525, -0.7248],
         [ 0.4201, -1.0890,  0.9348,  ...,  0.3074,  0.0193,  0.3120],
         ...,
         [ 0.1452, -0.0473, -0.3389,  ..., -0.1330, -0.3173, -0.5224],
         [ 0.0482,  0.7825,  0.1312,  ..., -1.0877, -0.0121, -0.3467],
         [ 0.0083,  0.1892, -0.5714,  ...,  0.0246, -0.2700, -0.2058]],

        [[-0.3661, -0.1557, -0.3091,  ..., -0.5143,  

torch.Size([20, 512, 768])

预训练模型结果得到一个 BaseModelOutputWithPoolingAndCrossAttentions 对象，其中包括 last_hidden_state 和 pooler_output 两个字段，关注 last_hidden_state 字段,该字段表示，20句话编码，每句话为 512 个词，每个词被抽象成一个 768维向量，

#### 2. 定义下游任务

In [18]:
import torch.nn as nn
class Model(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.fc = torch.nn.Linear(768,2)
  def forward(self, input_ids, attention_mask, token_type_ids):
    # 使用预训练模型抽取特征
    with torch.no_grad():
      out = pretrained(input_ids = input_ids,
        attention_mask = attention_mask,
        token_type_ids = token_type_ids)
    out = self.fc(out.last_hidden_state[:,0])
    m = nn.Softmax(dim=1)
    # out = out.nn.Softmax(dim=1)
    return m(out)

model = Model()
model.to(device)

# 试算
model(input_ids=var1, attention_mask=var2, token_type_ids=var3).shape

torch.Size([20, 2])

### 训练和测试
#### 1. 训练

In [30]:
from transformers import AdamW
from transformers.optimization import get_scheduler
def train():
  optimizer = AdamW( model.parameters(), lr = 5e-4)
  criterion = torch.nn.CrossEntropyLoss()
  scheduler = get_scheduler(
      name = "linear",
      num_warmup_steps = 0 ,
      num_training_steps = len(loader),
      optimizer = optimizer)

  model.train()
  for i,(input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    out = model(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    if i % 10 == 0:
      out = out.argmax(dim=1)
      accuracy = (out == labels).sum().item() / len(labels)
      lr = optimizer.state_dict()['param_groups'][0]['lr']
      print(i, loss.item(), lr, accuracy)

train()



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0.47570547461509705 0.0004989583333333334 0.85
10 0.46111875772476196 0.0004885416666666666 0.85
20 0.4383173882961273 0.00047812500000000003 0.9
30 0.4616750180721283 0.0004677083333333333 0.9
40 0.37962275743484497 0.00045729166666666666 0.95
50 0.3983020484447479 0.000446875 0.95
60 0.4038565158843994 0.00043645833333333335 0.95
70 0.45029473304748535 0.00042604166666666664 0.9
80 0.3700927495956421 0.00041562500000000003 1.0
90 0.48616546392440796 0.0004052083333333333 0.85
100 0.4818590581417084 0.00039479166666666666 0.85
110 0.4635850787162781 0.000384375 0.85
120 0.36107704043388367 0.00037395833333333335 1.0
130 0.5399025082588196 0.00036354166666666664 0.75
140 0.4271513819694519 0.00035312500000000003 0.9
150 0.4316268563270569 0.0003427083333333333 0.9
160 0.4070776104927063 0.00033229166666666666 0.9
170 0.4916042387485504 0.000321875 0.85
180 0.4272012710571289 0.00031145833333333335 0.85
190 0.4663541913032532 0.00030104166666666663 0.85
200 0.371951162815094 0.0002906

#### 2. 测试

In [31]:
def test():
  loader_test = torch.utils.data.DataLoader(
      dataset = Dataset('test'),
      batch_size = 32,
      collate_fn = collate_fn,
      shuffle = True,
      drop_last = True,
  )
  model.eval()
  total_list = []
  correct = 0
  total = 0
  for i,(input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
    # print(i)
    with torch.no_grad():
      out = model(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
      out = out.argmax(dim=1)
      correct += (out == labels).sum().item()
      total += len(labels)
    total_list.append(correct / total)
  return sum(total_list)/len(loader_test)
test()

0.8896187599854067

#### 3. 验证

In [32]:
def validation():
  loader_test = torch.utils.data.DataLoader(
      dataset = Dataset('validation'),
      batch_size = 32,
      collate_fn = collate_fn,
      shuffle = True,
      drop_last = True,
  )
  model.eval()
  total_list = []
  correct = 0
  total = 0
  for i,(input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
    # print(i)
    with torch.no_grad():
      out = model(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
      out = out.argmax(dim=1)
      correct += (out == labels).sum().item()
      total += len(labels)
    total_list.append(correct / total)
  return sum(total_list)/len(loader_test)
validation()

0.8848647069228274